In [1]:
import xarray as xr
import os
import hydrobm
import pandas as pd
from hydrobm.calculate import calc_bm
import numpy as np

### Inputs

In [2]:
base_dir ='camels-spat/camels-spat_inputs/'

output_dir = 'camels-spat/camels-spat_outputs/'



# Define calibration and validation periods (start and end)
calibration_range = ('1982-10-01', '1989-09-30')
validation_range  = ('1989-10-01', '2009-09-30')


# Specify the benchmarks and metrics to calculate
benchmarks = [
        # Streamflow benchmarks
        "mean_flow",
        "median_flow",
        "annual_mean_flow",
        "annual_median_flow",
        "monthly_mean_flow",
        "monthly_median_flow",
        "daily_mean_flow",
        "daily_median_flow",

        # Long-term rainfall-runoff ratio benchmarks
        "rainfall_runoff_ratio_to_all",
        "rainfall_runoff_ratio_to_annual",
        "rainfall_runoff_ratio_to_monthly",
        "rainfall_runoff_ratio_to_daily",
        "rainfall_runoff_ratio_to_timestep",

         # Short-term rainfall-runoff ratio benchmarks
        # "monthly_rainfall_runoff_ratio_to_monthly",
        # "monthly_rainfall_runoff_ratio_to_daily",
        # "monthly_rainfall_runoff_ratio_to_timestep",

        # Schaefli & Gupta (2007) benchmarks
        "scaled_precipitation_benchmark",  # equivalent to "rainfall_runoff_ratio_to_daily"
        "adjusted_precipitation_benchmark",
        "adjusted_smoothed_precipitation_benchmark",
     ]

### Pre-Processing

In [3]:
# Make output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

In [4]:
# List all directories that start with CAN or USA
folders = [f for f in os.listdir(base_dir)
           if os.path.isdir(os.path.join(base_dir, f)) and (f.startswith('CAN') or f.startswith('USA'))]

In [5]:
# Create paths to the 'input' folders
input_dirs = [os.path.join(base_dir, folder, 'input') for folder in folders]

In [6]:
input_dirs

['camels-spat/camels-spat_inputs/CAN_05MF001/input',
 'camels-spat/camels-spat_inputs/USA_01638480/input']

In [7]:
# Iterate through each path and open the .nc file containing 'input' in its name
for input_dir in input_dirs:
    for file in os.listdir(input_dir):
        # Find input NetCDF
        if file.endswith('.nc') and 'input' in file:

            
            # Save filename to variable
            station_id = file.split('_input')[0]

            # Create full filepath and read inputs
            file_path = os.path.join(input_dir, file)
            print(f"Opening {file_path}...")
            ds = xr.open_dataset(file_path)

            # =================================================
            # Pre-processing
            # Extract variables of interest (squeeze removes singleton lat/lon dims)
            bm_input = ds[['pr', 'q_obs', 'temp']].to_dataframe().reset_index()

            # Check for NaNs in key columns
            nan_mask = bm_input[['pr', 'q_obs', 'temp']].isna().any(axis=1)
            if nan_mask.any():
                print(f"Warning: NaNs detected in {station_id} input data!")
                print(bm_input.loc[nan_mask, ['time', 'pr', 'q_obs', 'temp']])
            
            # Drop lat/lon since they're constant
            bm_input = bm_input.drop(columns=['latitude', 'longitude'])
            
            # Set date/time as index
            bm_input = bm_input.set_index('time')


            # Vectorized mask creation
            bm_input['cal_mask'] = (bm_input.index >= pd.to_datetime(calibration_range[0])) & \
                                   (bm_input.index <= pd.to_datetime(calibration_range[1]))
            
            bm_input['val_mask'] = (bm_input.index >= pd.to_datetime(validation_range[0])) & \
                                   (bm_input.index <= pd.to_datetime(validation_range[1]))

            # Trim to only include calibration or validation periods
            bm_input = bm_input[bm_input['cal_mask'] | bm_input['val_mask']]

            # ===========================================
            # Calculate Benchmarks
            print('Calculating Benchmarks')
    
            # Calculate the benchmarks and scores
            benchmark_flows, scores = calc_bm(
                bm_input,
        
                # Time period selection
                bm_input['cal_mask'],
                val_mask=bm_input['val_mask'],
        
                # Variable names in 'data'
                precipitation="pr",
                streamflow="q_obs",
        
                # Benchmark choices
                benchmarks=benchmarks,
                metrics=['nse', 'kge'],
                optimization_method="brute_force",
        
                # Snow model inputs
                calc_snowmelt=True,
                temperature="temp",
                snowmelt_threshold=0.0,
                snowmelt_rate=3.0,
            )

            # Add lat and lon
            lat_value = ds['latitude'].values.item()  # safer than float()
            lon_value = ds['longitude'].values.item()
            
            benchmark_flows['latitude'] = lat_value
            benchmark_flows['longitude'] = lon_value


            # Reset indices for safe merging
            benchmark_flows = benchmark_flows.reset_index()
            bm_input_reset = bm_input.reset_index()
            
            # Select columns to merge: observed flow + masks
            bm_merge = bm_input_reset[['time', 'q_obs', 'cal_mask', 'val_mask']]
            
            # Merge on 'time'
            benchmark_flows = benchmark_flows.merge(bm_merge, on='time', how='left')

            # ===================================
            # Save benchmark_flows to CSV
            out_file = os.path.join(output_dir, f"{station_id}_BM.csv")
            benchmark_flows.to_csv(out_file)
            print(f"Saved benchmark flows to {out_file}")

            ds.close()

            

Opening camels-spat/camels-spat_inputs/CAN_05MF001/input/CAN_05MF001_input.nc...
Calculating Benchmarks
Saved benchmark flows to camels-spat/camels-spat_outputs/CAN_05MF001_BM.csv
Opening camels-spat/camels-spat_inputs/USA_01638480/input/USA_01638480_input.nc...
Calculating Benchmarks
Saved benchmark flows to camels-spat/camels-spat_outputs/USA_01638480_BM.csv


In [8]:
cal_set = bm_input.loc[bm_input['cal_mask']]

monthly_mean_q_cal = cal_set['q_obs'].groupby(cal_set.index.month).mean()
monthly_mean_p_cal = cal_set['pr'].groupby(cal_set.index.month).mean()

print("Monthly mean streamflow during calibration period:")
print(monthly_mean_q_cal)

# Check which months have all NaNs
all_nan_months = monthly_mean_q_cal[monthly_mean_q_cal.isna()]
if not all_nan_months.empty:
    print("\nMonths with all NaNs in q_obs during calibration period:")
    print(all_nan_months)
else:
    print("\nNo months have all NaNs in q_obs during calibration period.")

Monthly mean streamflow during calibration period:
time
1     0.742635
2     1.796651
3     1.875310
4     2.290759
5     2.158139
6     0.630837
7     0.676184
8     0.389720
9     0.170889
10    0.211408
11    0.661437
12    1.068285
Name: q_obs, dtype: float64

No months have all NaNs in q_obs during calibration period.


In [9]:
def monthly_rainfall_runoff_ratio_to_monthly(
    data, cal_mask, precipitation="precipitation", streamflow="streamflow"
):
    """Calculate the mean monthly rainfall-runoff ratio over the calculation period and
    use that as a predictor of runoff-from-precipitation for each month in the whole dataframe.

    Parameters
    ----------
    data : pandas DataFrame
        Input data containing precipitation and streamflow columns.
    cal_mask : pandas Series
        Boolean mask for the calculation period.
    precipitation : str, optional
        Name of the precipitation column in the input data. Default is ['precipitation'].
    streamflow : str, optional
        Name of the streamflow column in the input data. Default is ['streamflow'].

    Returns
    -------
    bm_vals: pandas DataSeries
        Monthly rainfall-runoff ratio values for the calculation period.
    qbm : pandas DataFrame
        Benchmark flow time series for the monthly rainfall-runoff ratio (RRR) benchmark model.
        Computed as mean monthly RRR multiplied by monthly mean precipitation.
    """

    cal_set = data.loc[cal_mask]
    monthly_mean_q = cal_set[streamflow].groupby(cal_set.index.month).mean()
    monthly_mean_p = cal_set[precipitation].groupby(cal_set.index.month).mean()
    bm_vals = monthly_mean_q / monthly_mean_p  # (at most) 12 rainfall-runoff ratios
    bm_vals = bm_vals.reindex(range(1, 13))  # fill missing months with NaN, does nothing if already 12
    qbm = pd.DataFrame({"bm_monthly_rainfall_runoff_ratio_to_monthly": np.nan}, index=data.index)
    for year in qbm.index.year.unique():  # for each year
        for month in qbm.loc[
            qbm.index.year == year
        ].index.month.unique():  # for each month we have in the index for that year (takes care of mising months)
            this_month = (data.index.year == year) & (data.index.month == month)
            mean_monthly_precip = data[precipitation].loc[this_month].mean()
            qbm.loc[this_month, "bm_monthly_rainfall_runoff_ratio_to_monthly"] = (
                bm_vals.loc[month] * mean_monthly_precip
            )
    return bm_vals, qbm

In [10]:
bm_input

,pr,q_obs,temp,cal_mask,val_mask,snow_depth,rain_plus_melt
time,,,,,,,
1982-10-01,0.000000,0.126642,16.712631,True,False,0.0,0.000000
1982-10-02,0.002036,0.105535,16.636405,True,False,0.0,0.002036
1982-10-03,0.000000,0.104479,17.174136,True,False,0.0,0.000000
1982-10-04,0.059440,0.092871,20.282466,True,False,0.0,0.059440
1982-10-05,0.108790,0.092871,20.113307,True,False,0.0,0.108790
...,...,...,...,...,...,...,...
2009-09-26,22.193546,0.061949,15.451477,False,True,0.0,22.193546
2009-09-27,3.887933,0.223734,17.235085,False,True,0.0,3.887933
2009-09-28,0.243591,0.216346,17.569997,False,True,0.0,0.243591


In [11]:
bm_vals, qbm= monthly_rainfall_runoff_ratio_to_monthly(bm_input, bm_input['cal_mask'], precipitation="pr", streamflow = 'q_obs')

In [12]:
bm_vals

time
1     0.428045
2     0.669150
3     0.730783
4     0.737129
5     0.504772
6     0.267241
7     0.268855
8     0.148519
9     0.076648
10    0.088704
11    0.171867
12    0.531247
dtype: float64